# Parcial 1 (Segunda parte)

Si realiza la ejecución de forma local no es necesario volver a instalar los siguientes paquetes, para los que desarrollen el documento en la nube recuerden primero instalr *findspark* y *pyspark* par poder acceder al core de **spark**.

In [ ]:
#!pip install -q findspark
#!pip install pyspark

Ejecute sólo una vez la siguiente celda para crear el Contexto de Spark y uselo para el desarrollo de todos los puntos del parcial

In [7]:
from pyspark import SparkConf, SparkContext
import findspark

findspark.init() ### Importantisimo
conf = SparkConf().setMaster("local[*]").setAppName("Parcial1")
sc = SparkContext(conf = conf)

## Ejercicio 1

In [8]:
import numpy as np
datos = [0.85, -3.9 ,  0.61,  2.01,  5.87, -3.81, -1.86, -2.26, -1.13,
       -2.6 ,  3.07, -6.85, -0.09,  3.36,  3.94,  0.15, -0.23, -1.16,
       -3.65,  0.24, -2.76, -1.82,  4.17, -2.39, -2.77,  2.48, -2.63,
       -1.99, -0.37, -0.31, -1.17,  1.06,  2.65,  0.84,  1.4 ,  1.89,
        3.78,  3.67, -8.51, -3.14, -6.22,  0.2 ,  2.9 , -0.  , -2.24,
        1.77,  6.01,  1.9 ,  0.38, -3.43,  2.23, -0.43, -5.34,  2.21,
       -1.34, -0.14,  0.89,  2.66, -5.53, -0.92,  4.75, -3.06, -0.07,
       -0.2 ,  0.04,  1.01,  0.01,  0.28,  3.59,  2.82,  5.54,  1.06,
       -4.95, -5.5 , -2.05,  1.59, -2.18, -4.95, -1.78,  0.82, -3.31,
        5.21, -1.42,  2.66,  4.1 , -0.32,  1.88, -1.33,  1.22, -0.46,
       -4.42, -1.72, -1.16, -1.47,  2.8 , -0.25,  2.54,  2.5 ,  2.41,
        1.35]

In [9]:
muestra = sc.parallelize(datos)

In [15]:
sumatoria = muestra.map(lambda x: x * x)\
              .reduce(lambda x, y: x + y)
cant = muestra.count()                
varianza = sumatoria / cant

print(media)

8.608911999999998


## Ejercicio 2

In [17]:
Patente = sc.textFile("./ocr-description.txt")

In [49]:
import re
PatenteFiltrada = Patente.map(lambda x: x.lower())\
                         .map(lambda x: re.sub(r'[^\w\s]', '', x))\
                         .flatMap(lambda x: x.split())\
                         .filter(lambda x: len(x) > 10)\
                         .map(lambda x: (x,1))\
                         .reduceByKey(lambda x, y: x + y)\
                         .sortBy(lambda x: x[1])

In [51]:
for palabra, cant in PatenteFiltrada.take(5):
    print(f"{palabra}: {cant}")

crossreference: 1
combinations: 1
crosssectional: 1
accommodating: 1
communicates: 1


## Ejercicio 3

In [41]:
ratings = sc.textFile("./ratings.dat")

In [42]:
def parseLines(x):
  x = x.split("::")[1:3]
  clave = x[0]
  valor = float(x[1])
  return(clave , valor)

In [44]:
parte1 = ratings.map(lambda x: x.split("::")[2])\
                .map(lambda rating: (rating, 1))\
                .reduceByKey(lambda x, y: x + y)
parte1.collect()

[('4', 348971), ('1', 56174), ('5', 226310), ('3', 261197), ('2', 107557)]

In [45]:
parte2 = ratings.map(parseLines)\
                   .mapValues(lambda x: (x, 1))\
                   .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                   .mapValues(lambda x: x[0] / x[1])
parte2.collect()

[('914', 4.154088050314465),
 ('3408', 3.863878326996198),
 ('2355', 3.854374633000587),
 ('1197', 4.3037100949094045),
 ('2804', 4.238905325443787),
 ('595', 3.8858490566037736),
 ('2791', 3.9711149624494513),
 ('3105', 3.7808823529411764),
 ('2797', 3.8558014755197854),
 ('2321', 3.6968911917098444),
 ('720', 4.426940639269406),
 ('48', 2.9764397905759163),
 ('1097', 3.9651828999559275),
 ('1545', 4.068493150684931),
 ('745', 4.52054794520548),
 ('1566', 3.279317697228145),
 ('588', 3.788304959289415),
 ('1907', 3.742857142857143),
 ('783', 3.223076923076923),
 ('1836', 3.311111111111111),
 ('2762', 4.406262708418057),
 ('150', 4.073541167066347),
 ('1', 4.146846413095811),
 ('1962', 3.8773006134969323),
 ('2692', 4.224813432835821),
 ('2028', 4.337353938937053),
 ('608', 4.254675686430561),
 ('1357', 3.8065693430656933),
 ('1537', 4.137142857142857),
 ('2194', 4.007985803016859),
 ('648', 3.432874918140144),
 ('2268', 3.93119698397738),
 ('3468', 4.24),
 ('1210', 4.022892819979188),

In [46]:
parte3 = parte2.filter(lambda x: x[1] >= 4.8)
parte3.collect()

[('787', 5.0),
 ('1830', 5.0),
 ('3607', 5.0),
 ('3280', 5.0),
 ('3245', 4.8),
 ('989', 5.0),
 ('3881', 5.0),
 ('3233', 5.0),
 ('3172', 5.0),
 ('3656', 5.0),
 ('3382', 5.0)]